# Chapter 2: Observer Pattern

## Background
In the world of software design, design patterns are solutions to common problems that occur in software design. These patterns are not ready-to-use code, but rather a description of how to solve a problem that can be used in many different situations. One such pattern is the Observer Pattern.

## Observer Pattern
The Observer Pattern is a software design pattern in which an object, called the **subject**, maintains a list of its dependents, called **observers**, and notifies them automatically of any state changes, usually by calling one of their methods. This pattern is mainly used to implement distributed event handling systems, in a "one-to-many" relationship.

## Purpose of this Chapter
In this chapter, we will be implementing the Observer Pattern in Python. The goal is to create a "Bulletin Board" system. In this system, the Bulletin Board (subject) will hold the data and multiple Display Elements (observers) will display this data in different formats. Whenever the data on the Bulletin Board changes, all Display Elements will automatically update to reflect these changes. This is a practical application of the Observer Pattern, demonstrating how it can be used to maintain consistency across multiple display elements that are all dependent on the same data source.

# Here is the 1st example from the book

In [1]:
from abc import ABC, abstractmethod

# Here we define the interfaces for the Observer pattern

# '_I' suffix is used to indicate that this is an interface
# Subject interface
class Subject_I(ABC):
    @abstractmethod
    def register_observer(self, observer):
        pass

    @abstractmethod
    def remove_observer(self, observer):
        pass

    @abstractmethod
    def notify_observers(self):
        pass

# Oberver interface
class Observer_I(ABC):
    @abstractmethod
    def update(self, subject):
        pass

# DisplayElement interface
class DisplayElement_I(ABC):
    @abstractmethod
    def display(self):
        pass


In [2]:
# Concrete Subject
class WeatherData(Subject_I):
    def __init__(self):
        self.observers = []

    def register_observer(self, observer):
        self.observers.append(observer)

    def remove_observer(self, observer):
        self.observers.remove(observer)

    def notify_observers(self):
        for observer in self.observers:
            observer.update(self)

    def measurements_changed(self):
        self.notify_observers()
    
    def set_measurements(self, temperature, humidity, pressure):
        self.temperature = temperature
        self.humidity = humidity
        self.pressure = pressure
        self.measurements_changed()

class currentConditionsDisplay(Observer_I, DisplayElement_I):
    def __init__(self, weatherData):
        self.weatherData = weatherData
        weatherData.register_observer(self)

    def update(self, weatherData):
        self.temperature = weatherData.temperature
        self.humidity = weatherData.humidity
        self.display()

    def display(self):
        print(f"Current conditions: {self.temperature}F degrees and {self.humidity}% humidity")

class statisticsDisplay(Observer_I, DisplayElement_I):
    def __init__(self, weatherData):
        self.weatherData = weatherData
        weatherData.register_observer(self)

    def update(self, weatherData):
        self.temperature = weatherData.temperature
        self.humidity = weatherData.humidity
        self.pressure = weatherData.pressure
        self.display()

    def display(self):
        print(f"Statistics: {self.temperature}F degrees and {self.humidity}% humidity and {self.pressure} pressure")



In [6]:
# Main

class WeatherStation:
    def __init__(self):
        self.weatherData = WeatherData()
        self.currentDisplay = currentConditionsDisplay(self.weatherData)
        self.statisticsDisplay = statisticsDisplay(self.weatherData)

    def start_work(self):
        print("Weather Station started working")
        print("\nThis is the 1st broadcast:(include CurrentConditions and Statistics)")
        self.weatherData.set_measurements(80, 65, 30.4)
        print("\nThis is the 2nd broadcast:(include CurrentConditions and Statistics)")
        self.weatherData.set_measurements(82, 70, 29.2)
        self.weatherData.remove_observer(self.currentDisplay)
        print("\nThis is the 3rd broadcast:(only include Statistics)")
        self.weatherData.set_measurements(78, 90, 29.2)


myWeatherStation = WeatherStation()
myWeatherStation.start_work()

Weather Station started working

This is the 1st broadcast:(include CurrentConditions and Statistics)
Current conditions: 80F degrees and 65% humidity
Statistics: 80F degrees and 65% humidity and 30.4 pressure

This is the 2nd broadcast:(include CurrentConditions and Statistics)
Current conditions: 82F degrees and 70% humidity
Statistics: 82F degrees and 70% humidity and 29.2 pressure

This is the 3rd broadcast:(only include Statistics)
Statistics: 78F degrees and 90% humidity and 29.2 pressure


# How about use Measurements rather than the specific data?

In the original code, we have specific measurements like temperature, humidity, and pressure. However, in real-world scenarios, we might have new measurements added over time. To make our code more flexible and easier to maintain, we can abstract these measurements into an interface, which we'll call `Measurements_I`. 

By doing this, we can easily add new measurements without changing the existing code. This is an application of the Open-Closed Principle, which states that software entities should be open for extension, but closed for modification. 

Here's how we can refactor the code:

In [10]:
from abc import ABC, abstractmethod
from typing import NamedTuple, Tuple

class Measurements(NamedTuple):
    temperature: float = 0
    humidity: float = 0
    pressure: float = 0
    geo : Tuple[float, float] = (0, 0)

class Measurements_I(ABC):
    @abstractmethod
    def set_measurements(self, measurements: Measurements):
        pass

class WeatherData(Subject_I, Measurements_I):
    def __init__(self):
        self.observers = []
        self.measurements = Measurements()

    def register_observer(self, observer):
        self.observers.append(observer)

    def remove_observer(self, observer):
        self.observers.remove(observer)

    def notify_observers(self):
        for observer in self.observers:
            observer.update(self.measurements)

    def measurements_changed(self):
        self.notify_observers()

    def set_measurements(self, measurements: Measurements):
        self.measurements = measurements
        self.measurements_changed()

class CurrentConditionsDisplay(Observer_I, DisplayElement_I):
    def __init__(self, weatherData: WeatherData):
        self.weatherData = weatherData
        weatherData.register_observer(self)

    def update(self, measurements: Measurements):
        self.temperature = measurements.temperature
        self.humidity = measurements.humidity
        self.display()

    def display(self):
        print(f"Current conditions: {self.temperature}F degrees and {self.humidity}% humidity")

class StatisticsDisplay(Observer_I, DisplayElement_I):
    def __init__(self, weatherData: WeatherData):
        self.weatherData = weatherData
        weatherData.register_observer(self)

    def update(self, measurements: Measurements):
        self.temperature = measurements.temperature
        self.humidity = measurements.humidity
        self.pressure = measurements.pressure
        self.display()

    def display(self):
        print(f"Statistics: {self.temperature}F degrees and {self.humidity}% humidity and {self.pressure} pressure")

class ForecastDisplay(Observer_I, DisplayElement_I):
    def __init__(self, weatherData: WeatherData):
        self.weatherData = weatherData
        weatherData.register_observer(self)

    def update(self, measurements: Measurements):
        self.pressure = measurements.pressure
        self.geo = measurements.geo
        self.display()

    def display(self):
        print(f"In {self.geo[0], self.geo[1]} the pressure will become {self.pressure} ")


class WeatherStation:
    def __init__(self):
        self.weatherData = WeatherData()
        self.currentDisplay = currentConditionsDisplay(self.weatherData)
        self.statisticsDisplay = statisticsDisplay(self.weatherData)

    def start_work(self):
        print("Weather Station started working")
        measurements = Measurements(78, 90, 29.2)

        print("\nThis is the 1st broadcast:(include CurrentConditions and Statistics)")
        self.weatherData.set_measurements(measurements)

        print("\nThis is the 2nd broadcast:(include CurrentConditions and Statistics)")
        self.weatherData.set_measurements(measurements)
        self.weatherData.remove_observer(self.currentDisplay)

        print("\nThis is the 3rd broadcast:(only include Statistics)")
        self.weatherData.set_measurements(measurements)

        print("\nThis is the 4th broadcast:(include Statistics and Forecast)")
        measurements = Measurements(78, 90, 29.2, (40.7128, 74.0060))
        self.forrecastDisplay = ForecastDisplay(self.weatherData)
        self.weatherData.set_measurements(measurements)


myWeatherStation = WeatherStation()
myWeatherStation.start_work()

Weather Station started working

This is the 1st broadcast:(include CurrentConditions and Statistics)
Current conditions: 78F degrees and 90% humidity
Statistics: 78F degrees and 90% humidity and 29.2 pressure

This is the 2nd broadcast:(include CurrentConditions and Statistics)
Current conditions: 78F degrees and 90% humidity
Statistics: 78F degrees and 90% humidity and 29.2 pressure

This is the 3rd broadcast:(only include Statistics)
Statistics: 78F degrees and 90% humidity and 29.2 pressure

This is the 4th broadcast:(include Statistics and Forecast)
Statistics: 78F degrees and 90% humidity and 29.2 pressure
In (40.7128, 74.006) the pressure will become 29.2 
